<h1>Buscando dados da internet</h1>

<h3>Laboratório de Coletas, Preparação e Análise de Dados.</h3>
Desenvolvendo um crawler básico.

Prof. Luan Garcia

Baseado no material de Richard Mitchel
https://github.com/REMitchell/python-scraping

Com a biblioteca Beautiful Soup vimos como é possível tratar documentos HTML como objetos no modelo DOM, buscando informação pelas *tags* e seus atributos. Porém, para chegarmos ao ponto de processar um documento HTML precisamos realizar o download do documento a partir do seu servidor web e como descobrir novos documentos a partir de um link de origem.

Neste notebook, veremos como é possível fazer requisições HTTP utilizando a biblioteca urllib e como lidar com alguns problemas básicos que podem acontecer quando estamos trabalhando com um crawler.


Primeiro passo é importar o módulo request da biblioteca urllib. Não é necessário instalar, pois é uma biblioteca nativa do Python.

In [ ]:
from urllib.request import urlopen


Para fazer o download do documento web, basta sabermos o endereço do documento e chamar o método read() do **urlopen**.

In [ ]:
html = urlopen('http://pythonscraping.com/pages/page1.html')
print(html.read())

Para podermos tratar o documento utilizando a biblioteca Beautiful podemos passar diretamente o documento carregado com através do HTTP e depois manipular conforme o necessário.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page1.html')
bs = BeautifulSoup(html.read(), 'html.parser')
print(bs.h1)

Um problema bastante comum é o site que estamos tentando acessar não estar disponível, seja porque erramos o endereço, seja porque o servidor está fora do ar.

Um solução simples é garantir que tenhamos um documento para tratar utilizando exceções.

In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

try:
    html = urlopen("https://pythonscrapingthisurldoesnotexist.com")
except HTTPError as e:
    print("The server returned an HTTP error")
except URLError as e:
    print("The server could not be found!")
else:
    print(html.read())

O objetivo de um crawler é descobrir, a partir de um link raíz, outros documentos que estejam linkados de alguma forma.

Uma maneira simples de realizar isto é filtrando pelas tags de links de um documento HTML. Com isto, encontramos todos os links dentro de um documento.

Abaixo, um exemplo utilizando como raíz a página do ator Kevin Bacon na Wikipedia.

In [ ]:
html = urlopen('http://en.wikipedia.org/wiki/Kevin_Bacon')
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find_all('a'):
    if 'href' in link.attrs:
        print(link.attrs['href'])

Evidentemente, nem todos links serão de nosso interesse. Podemos filtrar apenas os links que nos interessam procurando por algum padrão no endereço e utilizar uma expressão regular para realizar o filtro.

Abaixo, vamos filtrar apenas links para outros artigos da wiki, ignorando âncoras, links para arquivos, etc. Faremos isso nos aproveitando de conhecimento de como um verbete na wiki é organizado. Todos links de artigos estarão sempre dentro da tag **div** que contém um atributo de **id** com valor **'bodyContent'**. Além disso, todo link de verbete necessariamente começa com o endereço "/wiki/" e não possui ":" no endereço.

In [ ]:
import re

html = urlopen('http://en.wikipedia.org/wiki/Kevin_Bacon')
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find('div', {'id':'bodyContent'}).find_all(
    'a', href=re.compile('^(/wiki/)((?!:).)*$')):
    if 'href' in link.attrs:
        print(link.attrs['href'])

Podemos generalizar este código em forma de uma função **getLinks()**. Isto possibilitará que busquemos os links de qualquer verbete da wiki.

In [ ]:
import re

def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')
    return bs.find('div', {'id':'bodyContent'}).find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks('/wiki/Kevin_Bacon')

for i in range(len(links)):
    newArticle = links[i].attrs['href']
    print(newArticle)

A função **getLinks()** anterior funciona se quisermos encontrar todos os links de uma única página, porém, se quisermos fazer um crawler efetivo, precisamos procurar por páginas linkadas dentro de outras páginas de forma recursiva. Podemos fazer isso chamando a nosa própria função de procurar link de forma recursiva.

In [ ]:
pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a', href=re.compile('^(/wiki/)((?!:).)*$')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                #Encontramos um link para uma página nva
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)


Experimente executar o comando a seguir e veja o que acontece. 

In [ ]:
getLinks('')

O que aconteceu? Será que isto é um problema? Se for, como podemos solucioná-lo?